# Сравнение моделей

## Импорты и генерация данных

In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib.pylab as plt
%matplotlib inline
import pandas as pd
from statsmodels.regression.linear_model import OLS

In [ ]:
X = np.random.randn(1000, 2)
w = np.array([1, 1])
y = np.dot(X, w) + np.random.randn(1000) * 0.1
noise = np.random.randn(1000)

In [ ]:
plt.scatter(noise, y)

In [ ]:
plt.scatter(X[:, 0], y)

In [ ]:
plt.scatter(X[:, 1], y)

In [ ]:
plt.scatter(np.sum(X, 1), y)

## Линейная регрессия

### Модель без шума из одного признака

#### Инициализация через аргументы

In [ ]:
X.shape, y.shape

In [ ]:
simple_model = OLS(y, X[:, 0]).fit()
simple_model.summary()

#### Инициализация через формулы

In [ ]:
simple_model = OLS.from_formula('y ~ x1 - 1', {'y' : y, 'x1' : X[:, 0]}).fit()
simple_model.summary()

### Модель шума

In [ ]:
noise_model = OLS(y, noise).fit()
noise_model.summary()

### Модель без шума из двух признаков

In [ ]:
complex_model = OLS.from_formula('y ~ x1 + x2 - 1', {'y' :  y, 'x1' : X[:, 0], 'x2' : X[:, 1]}).fit()
complex_model.summary()

### Сравнение вложенных моделей

Каждый тест возвращает:

- значение статистики (так как, мы исследуем важность только одного признака, F-критерий эквивалентен критерию Стьюдента для двусторонней альтернативы)
- p-value
- разница в количестве степеней свободы между моделями

#### Критерий Фишера

In [ ]:
complex_model.compare_f_test(simple_model)

#### Критерий отношения правдоподобия ($H_0$: равенство правдоподобий двух моделей)

In [ ]:
complex_model.compare_lr_test(simple_model)

#### Критерий Бройша-Пагана (множителей Лагранжа)

In [ ]:
complex_model.compare_lm_test(simple_model)

Нулевая гипотеза об отсутствии влияния второго признака на целевую переменную отвергается

### Добавим шум

In [ ]:
from statsmodels.regression.linear_model import OLS
complex_model2 = OLS.from_formula('y ~ x1 + x2 + noise - 1',
                                  {
                                      'y' : y,
                                      'noise' : noise,
                                      'x1' : X[:, 0],
                                      'x2' : X[:, 1]
                                  }).fit()
complex_model2.summary()

In [ ]:
complex_model2.compare_lm_test(complex_model)

In [ ]:
complex_model2.compare_lm_test(noise_model)

In [ ]:
complex_model.summary()

In [ ]:
complex_model.summary()

In [ ]:
noise_model.summary()

## Выбор моделей: случай невложенных моделей

In [ ]:
X = np.random.randn(100)
y = X**2 + np.random.randn(100) * 0.1
plt.scatter(X, y)

In [ ]:
plt.scatter(X ** 2, y)

### Линейная модель

In [ ]:
model1 = OLS(y, X).fit()
model1.summary()

### Квадратичная модель

In [ ]:
model2 = OLS(y, X ** 2).fit()
model2.summary()

### Предсказания моделей

In [ ]:
y_predicted1 = model1.predict(X)
y_predicted2 = model2.predict(X ** 2)

In [ ]:
plt.scatter(y_predicted1, y - y_predicted1)
plt.xlabel('$\hat y_1$')
_ = plt.ylabel('$\epsilon$')

In [ ]:
plt.scatter(y_predicted2, y)
plt.xlabel('$\hat y_2$')
plt.ylabel('y')

#### Критерий Давидсона-Маккиннона

In [ ]:
data = {'y' : y, 'y1' : y_predicted1, 'y2': y_predicted2, 'X' : X, 'X2' : X ** 2}
model1_with_y_2 = # ╰( ͡° ͜ʖ ͡° )-──☆*:・ﾟваш код
model2_with_y_1 = # ╰( ͡° ͜ʖ ͡° )-──☆*:・ﾟваш код

In [ ]:
model1_with_y_2.summary()

In [ ]:
model2_with_y_1.summary()

## Кодирование категориальных переменных

In [ ]:
data = [1] * 10 + [2] * 7 + [3] * 5
np.random.shuffle(data)
data = np.array(data)
data

### dummy

In [ ]:
from patsy.contrasts import Treatment
levels = [1, 2, 3]
contrast = Treatment().code_without_intercept(levels)
print(contrast.matrix)

In [ ]:
contrast.matrix[data - 1]

#### deviation

In [ ]:
from patsy.contrasts import Sum
contrast = Sum().code_without_intercept(levels)
print(contrast.matrix)

In [ ]:
contrast.matrix[data - 1]

# Метод Бокса-Кокса

In [ ]:
rs = np.random.RandomState(42)
y = rs.randn(100)
print(np.array(y > 1).sum())

Изменим хвост нормального распределения

In [ ]:
_ = plt.hist(y)

In [ ]:
tails = (y) > 1
y[tails] *= 1.45

y = y - np.min(y) + 1
_ = plt.hist(y)

In [ ]:
def add_titlebox(ax, text):
    ax.text(.55, .8, text,
            horizontalalignment='center',
            transform=ax.transAxes,
            bbox=dict(facecolor='white', alpha=0.6),
            fontsize=12.5)
    return ax

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(9, 7))
lambdas = [-5, -2, -1, -0.5, 0, 0.5, 1, 2, 5]
for i, l in enumerate(lambdas):
    axes[i // 3, i % 3].hist(st.boxcox(y, l))
    add_titlebox(axes[i // 3, i % 3], 'lambda = ' + str(l))

In [ ]:
for l in np.arange(-2.0,  2.0, 0.1):    
    print(l, st.shapiro(st.boxcox(y, l)))

In [ ]:
bc, l = st.boxcox(y)
print(f'optimal lambda = {l}')
print('Box-Cox transformed array:')
print(bc)

## Гетероскедастичность

In [ ]:
rs = np.random.RandomState(42)
X = rs.randn(100)
X.sort()
error = rs.randn(100) * 0.1 * np.arange(100)
y = X + error
y = y - np.min(y) + 1
plt.scatter(X, y)
plt.xlabel('X')
plt.ylabel('y')
print(y.min())

In [ ]:
model = OLS(y, X).fit()
predicted = model.predict(X)
plt.scatter(predicted, y - predicted)
plt.xlabel('$\hat y$')
_ = plt.ylabel('$\epsilon$')

### Преобразование Бокса-Кокса вручную

In [ ]:
def W(y, lam):
    # ╰( ͡° ͜ʖ ͡° )-──☆*:・ﾟваш код

И проверим

In [ ]:
for l in [-2, -1, -0.5, 0, 0.5, 1, 2]:
    print (np.sum(W(y, l) - st.boxcox(y, l)))

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(9, 7))
fig.suptitle('Residual from predicted y')

model = OLS(y, X).fit()
predicted = model.predict(X)
axes[0, 0].scatter(predicted, y - predicted )
axes[0, 0].set_ylim((-2, 50))
add_titlebox(axes[0, 0], 'original')
print(max(y))
    
lams = [-15, -10, -5, -1, -0.5, 0, 0.5, 1]
for i, l in enumerate(lams):
    i += 1
    model = OLS(W(y, l), X).fit()
    predicted = model.predict(X)
    axes[i // 3, i % 3].scatter(predicted, W(y, l) - predicted)
    add_titlebox(axes[i // 3, i % 3], 'lambda = ' + str(l))
    axes[i // 3, i % 3].set_ylim((0, 50))

In [ ]:
lams = np.arange(-5, 5, 0.1)
r = []
for l in lams:
    model = OLS(W(y, l), X).fit()
    r.append(np.log(model.mse_resid))
plt.plot(lams, r)
plt.xlabel('$\lambda$')
plt.ylabel('mse')
plt.title('Mse from lambda ')
lams[np.argmin(r)]

### Before

In [ ]:
model = OLS(y, X).fit()
print(model.mse_resid)

In [ ]:
model.summary()

### After

In [ ]:
bc, l = st.boxcox(y)
model = OLS(bc, X).fit()
print(model.mse_resid)

In [ ]:
model.summary()

In [ ]:
model = OLS(W(y, -5), X).fit()
print(model.mse_resid)

In [ ]:
model.summary()